## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,22.0752,-159.3190,80.58,76,60,7.00,broken clouds
1,1,Klaksvik,FO,62.2266,-6.5890,45.52,98,100,12.86,overcast clouds
2,2,Albany,US,42.6001,-73.9662,40.03,88,100,2.59,overcast clouds
3,3,Yellowknife,CA,62.4560,-114.3525,23.41,66,99,13.00,overcast clouds
4,4,Marsa Matruh,EG,31.3525,27.2453,63.34,69,0,9.22,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
user_max_temp = float(input("What's the hottest you'd like the temperature? "))
user_min_temp = float(input("What's the coldest you'd want the temperature? "))


What's the hottest you'd like the temperature? 90
What's the coldest you'd want the temperature? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_df["Max Temp"] = pd.to_numeric(city_data_df["Max Temp"])

preferred_cities = city_data_df.loc[(city_data_df["Max Temp"] <= user_max_temp) & (city_data_df["Max Temp"] >= user_min_temp)]

preferred_cities


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,22.0752,-159.3190,80.58,76,60,7.00,broken clouds
27,27,Bilma,NE,18.6853,12.9164,85.10,5,11,5.23,few clouds
43,43,Hope,US,33.6671,-93.5916,80.65,45,2,6.98,clear sky
53,53,Hithadhoo,MV,-0.6000,73.0833,82.74,70,32,5.48,scattered clouds
58,58,Avarua,CK,-21.2078,-159.7750,80.65,78,20,5.75,few clouds
...,...,...,...,...,...,...,...,...,...,...
621,621,Calamar,CO,10.2511,-74.9146,86.70,62,100,8.52,overcast clouds
628,628,Makakilo City,US,21.3469,-158.0858,82.72,48,0,20.71,clear sky
635,635,Dhidhdhoo,MV,6.8833,73.1000,83.03,71,6,4.47,clear sky
637,637,Cairns,AU,-16.9167,145.7667,82.06,88,75,1.14,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities.isnull().sum()

Unnamed: 0             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities.drop("Unnamed: 0", axis=1)
clean_df = clean_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,80.58,broken clouds,22.0752,-159.3190,
27,Bilma,NE,85.10,few clouds,18.6853,12.9164,
43,Hope,US,80.65,clear sky,33.6671,-93.5916,
53,Hithadhoo,MV,82.74,scattered clouds,-0.6000,73.0833,
58,Avarua,CK,80.65,few clouds,-21.2078,-159.7750,
66,Butaritari,KI,81.46,overcast clouds,3.0707,172.7902,
72,Miraflores,MX,81.25,overcast clouds,19.2167,-98.8083,
73,Kieta,PG,82.38,broken clouds,-6.2167,155.6333,
81,Alyangula,AU,82.35,scattered clouds,-13.8483,136.4192,
93,San Patricio,US,86.18,clear sky,28.0170,-97.5169,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key, 
    "location": []
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    gLat = row["Lat"]
    gLng = row["Lng"]
    params["location"] = f"{gLat}, {gLng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    gUrl = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    

    # 6e. Make request and retrieve the JSON data from the search. 
    nearby_lodging_json = requests.get(gUrl, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = nearby_lodging_json["results"][0]["name"]
    except(IndexError):
        print("No hotel in this city's radius. Skipping...")
        

No hotel in this city's radius. Skipping...
No hotel in this city's radius. Skipping...
No hotel in this city's radius. Skipping...
No hotel in this city's radius. Skipping...
No hotel in this city's radius. Skipping...
No hotel in this city's radius. Skipping...
No hotel in this city's radius. Skipping...
No hotel in this city's radius. Skipping...


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,80.58,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
27,Bilma,NE,85.10,few clouds,18.6853,12.9164,Expeditions Ténére voyages
43,Hope,US,80.65,clear sky,33.6671,-93.5916,Best Western Hope
53,Hithadhoo,MV,82.74,scattered clouds,-0.6000,73.0833,Scoop Guest House
58,Avarua,CK,80.65,few clouds,-21.2078,-159.7750,Paradise Inn
...,...,...,...,...,...,...,...
621,Calamar,CO,86.70,overcast clouds,10.2511,-74.9146,Hostal Katyalex
628,Makakilo City,US,82.72,clear sky,21.3469,-158.0858,Marriott's Ko Olina Beach Club
635,Dhidhdhoo,MV,83.03,clear sky,6.8833,73.1000,Dhidhoo beach
637,Cairns,AU,82.06,broken clouds,-16.9167,145.7667,Mantra Trilogy Cairns


In [11]:
# 8a. Create the output File (CSV)
output_data_file = clean_hotel_df.to_csv('WeatherPy_vacation.csv')
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

'City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name\r\n0,Kapaa,US,80.58,broken clouds,22.0752,-159.319,Sheraton Kauai Coconut Beach Resort\r\n27,Bilma,NE,85.1,few clouds,18.6853,12.9164,Expeditions Ténére voyages\r\n43,Hope,US,80.65,clear sky,33.6671,-93.5916,Best Western Hope\r\n53,Hithadhoo,MV,82.74,scattered clouds,-0.6,73.0833,Scoop Guest House\r\n58,Avarua,CK,80.65,few clouds,-21.2078,-159.775,Paradise Inn\r\n66,Butaritari,KI,81.46,overcast clouds,3.0707,172.7902,Isles Sunset Lodge\r\n72,Miraflores,MX,81.25,overcast clouds,19.2167,-98.8083,Hotel Mansión\r\n73,Kieta,PG,82.38,broken clouds,-6.2167,155.6333,Uruna Bay Retreat\r\n81,Alyangula,AU,82.35,scattered clouds,-13.8483,136.4192,Groote Eylandt Lodge\r\n93,San Patricio,US,86.18,clear sky,28.017,-97.5169,"Motel 6 Sinton, TX"\r\n94,Kattivakkam,IN,84.29,broken clouds,13.2167,80.3167,Euronova Freezer\r\n100,Samarai,PG,81.3,broken clouds,-10.6167,150.6667,Nuli Sapi\r\n116,Lorengau,PG,81.14,overcast clouds,-2.0226,147

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weatherd</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))